In [1]:
import mne
import glob
import numpy as np
import pandas as pd


In [2]:
files = glob.glob('eeg_data_task_rest/*.edf')
len(files)

72

In [3]:
rest_files = [i for i in files if i[-5] == '1']
task_files = [i for i in files if i[-5] == '2']
len(rest_files), len(task_files)

(36, 36)

In [4]:
def read_data(file_path):
    data = mne.io.read_raw_edf(file_path, preload=True)
    data.set_eeg_reference()
    data.filter(l_freq = 0.5, h_freq = 50)
    epochs = mne.make_fixed_length_epochs(data, duration =5, overlap = 1)
    array = epochs.get_data()
    return array

In [5]:
%%capture
rest_epochs_array = [read_data(i) for i in rest_files]
task_epochs_array = [read_data(i) for i in task_files]

In [6]:
rest_epochs_array[12].shape

(45, 21, 2500)

In [7]:
task_epochs_array[34].shape

(15, 21, 2500)

In [8]:
rest_epochs_label = [len(i)*[0] for i in rest_epochs_array]
task_epochs_label = [len(i)*[1] for i in task_epochs_array]

In [9]:
data_list = rest_epochs_array+task_epochs_array
label_list = rest_epochs_label+task_epochs_label

In [10]:
len(label_list[4])

42

In [11]:
data_array = np.vstack(data_list)
label_array = np.hstack(label_list)

In [12]:
data_array.shape

(2132, 21, 2500)

In [13]:
label_array.shape

(2132,)

In [14]:
data_array = np.moveaxis(data_array, 1, 2)
data_array.shape

(2132, 2500, 21)

In [15]:
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, LeakyReLU, MaxPooling1D, GlobalAveragePooling1D, Dense, Dropout, AveragePooling1D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.backend import clear_session

def EEGNetModel(input_shape):
    clear_session()
    inputs = Input(shape=input_shape)
    #Block1
    x = Conv1D(8, 32, activation='relu')(inputs)
    x = MaxPooling1D(2)(x)
    x = Dropout(0.2)(x)

    #Block2
    x = Conv1D(16, 32, activation='relu')(x)
    x = MaxPooling1D(2)(x)
    x = Dropout(0.2)(x)

    #Block3
    x = Conv1D(32, 32, activation='relu')(x)
    x = MaxPooling1D(2)(x)
    x = Dropout(0.2)(x)

    #Flatten and Dense Layers
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    #Define the model
    model = Model(inputs = inputs, outputs=outputs)

    #compile
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

model = EEGNetModel((2500, 21))
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 2500, 21)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 2469, 8)        │         5,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 1234, 8)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1234, 8)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 1203, 16)       │         4,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 601, 16)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 601, 16)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 570, 32)        │        16,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 285, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 285, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 9120)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,167,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,193,529 (4.55 MB)

 Trainable params: 1,193,529 (4.55 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_array, label_array, test_size=0.3)


In [17]:
X_train.shape

(1492, 2500, 21)

Apply StandardScalar before training

In [23]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(X_train.shape[0], -1)).reshape(X_train.shape)
X_test_scaled = scaler.transform(X_test.reshape(X_test.shape[0], -1)).reshape(X_test.shape)


In [147]:
#without scaling
model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=32)

Epoch 1/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 7s 84ms/step - accuracy: 0.7329 - loss: 0.5928 - val_accuracy: 0.7297 - val_loss: 0.5881
Epoch 2/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step - accuracy: 0.7433 - loss: 0.5756 - val_accuracy: 0.7297 - val_loss: 0.5893
Epoch 3/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 68ms/step - accuracy: 0.7652 - loss: 0.5484 - val_accuracy: 0.7297 - val_loss: 0.5837
Epoch 4/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - accuracy: 0.7373 - loss: 0.5765 - val_accuracy: 0.7297 - val_loss: 0.5839
Epoch 5/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 70ms/step - accuracy: 0.7645 - loss: 0.5496 - val_accuracy: 0.7297 - val_loss: 0.5938
Epoch 6/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step - accuracy: 0.7565 - loss: 0.5680 - val_accuracy: 0.7297 - val_loss: 0.5839
Epoch 7/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 71ms/step - accuracy: 0.7584 - loss: 0.5629 - val_accuracy: 0.7297 - val_loss: 0.5848
Epoch 8/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 70ms/step - accuracy: 0.7591 - loss: 0.5565 - val_accuracy: 0.7297 - v

In [25]:
#with scaling
model_for_scaled = EEGNetModel((2500, 21))
model_for_scaled.fit(X_train_scaled, y_train, epochs = 5, validation_data = (X_test_scaled, y_test), batch_size = 32)

Epoch 1/5
47/47 ━━━━━━━━━━━━━━━━━━━━ 6s 79ms/step - accuracy: 0.6634 - loss: 0.9590 - val_accuracy: 0.7344 - val_loss: 0.5632
Epoch 2/5
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - accuracy: 0.7580 - loss: 0.5493 - val_accuracy: 0.7344 - val_loss: 0.5826
Epoch 3/5
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - accuracy: 0.7626 - loss: 0.5288 - val_accuracy: 0.7344 - val_loss: 0.5297
Epoch 4/5
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - accuracy: 0.7477 - loss: 0.4938 - val_accuracy: 0.7531 - val_loss: 0.5041
Epoch 5/5
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - accuracy: 0.7912 - loss: 0.4339 - val_accuracy: 0.7859 - val_loss: 0.4670


We can see that we fot 78.6% sccuracy for our scaled model for insubject split

Lets check for cross test

In [26]:
train_rest_files = rest_files[:10]+rest_files[15:32]
train_task_files = task_files[:10]+task_files[15:32]

In [27]:
%%capture
train_rest_epochs_array = [read_data(i) for i in train_rest_files]
train_task_epochs_array = [read_data(i) for i in train_task_files]

In [28]:
train_rest_epochs_label = [len(i)*[0] for i in train_rest_epochs_array]
train_task_epochs_label = [len(i)*[1] for i in train_task_epochs_array]

In [29]:
train_data_list = train_rest_epochs_array+train_task_epochs_array
train_label_list = train_rest_epochs_label+train_task_epochs_label

In [30]:
from sklearn.utils import shuffle

In [31]:
train_data_array = np.vstack(train_data_list)
train_label_array = np.hstack(train_label_list)
train_data_array = np.moveaxis(train_data_array, 1, 2)
X, y = shuffle(train_data_array, train_label_array)

In [32]:
%%capture
val_rest_files = rest_files[10:18]+rest_files[33:]
val_task_files = task_files[10:18]+task_files[33:]

val_rest_epochs_array = [read_data(i) for i in val_rest_files]
val_task_epochs_array = [read_data(i) for i in val_task_files]

val_rest_epochs_label = [len(i)*[0] for i in val_rest_epochs_array]
val_task_epochs_label = [len(i)*[1] for i in val_task_epochs_array]

val_data_list = val_rest_epochs_array+val_task_epochs_array
val_label_list = val_rest_epochs_label+val_task_epochs_label

val_data_array = np.vstack(val_data_list)
val_label_array = np.hstack(val_label_list)
val_data_array = np.moveaxis(val_data_array, 1, 2)

In [33]:
model_cross = EEGNetModel((2500, 21))

In [156]:
model_cross.fit(X, y, epochs=10, batch_size=64, validation_data=(val_data_array, val_label_array))

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 150ms/step - accuracy: 0.6904 - loss: 0.6267 - val_accuracy: 0.7504 - val_loss: 0.5634
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 120ms/step - accuracy: 0.7319 - loss: 0.5919 - val_accuracy: 0.7504 - val_loss: 0.5628
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 123ms/step - accuracy: 0.7325 - loss: 0.5813 - val_accuracy: 0.7504 - val_loss: 0.5622
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 121ms/step - accuracy: 0.7560 - loss: 0.5555 - val_accuracy: 0.7504 - val_loss: 0.5619
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 122ms/step - accuracy: 0.7446 - loss: 0.5785 - val_accuracy: 0.7504 - val_loss: 0.5719
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 116ms/step - accuracy: 0.7399 - loss: 0.5811 - val_accuracy: 0.7504 - val_loss: 0.5656
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 118ms/step - accuracy: 0.7399 - loss: 0.5777 - val_accuracy: 0.7504 - val_loss: 0.5747
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 116ms/step - accuracy: 0.7374 - loss: 0.5857 - val_accuracy: 0.

Cross test yields result as 75% accuracy

In [35]:
#after scaling
scaler2 = StandardScaler()
X_scaled = scaler2.fit_transform(X.reshape(X.shape[0], -1)).reshape(X.shape)
val_data_array_scaled = scaler2.transform(val_data_array.reshape(val_data_array.shape[0], -1)).reshape(val_data_array.shape)
model_cross_scaled = EEGNetModel((2500, 21))
model_cross_scaled.fit(X, y, epochs = 10, validation_data = (val_data_array_scaled, val_label_array), batch_size = 32)

Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 7s 79ms/step - accuracy: 0.7273 - loss: 0.5998 - val_accuracy: 0.7504 - val_loss: 12.0580
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 71ms/step - accuracy: 0.7566 - loss: 0.5633 - val_accuracy: 0.7504 - val_loss: 7.0239
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 68ms/step - accuracy: 0.7582 - loss: 0.5573 - val_accuracy: 0.7504 - val_loss: 5.6196
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - accuracy: 0.7465 - loss: 0.5780 - val_accuracy: 0.7504 - val_loss: 4.0954
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - accuracy: 0.7525 - loss: 0.5644 - val_accuracy: 0.7504 - val_loss: 2.3899
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - accuracy: 0.7399 - loss: 0.5796 - val_accuracy: 0.7504 - val_loss: 3.4653
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - accuracy: 0.7495 - loss: 0.5641 - val_accuracy: 0.7504 - val_loss: 3.1119
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - accuracy: 0.7439 - loss: 0.5726 - val_accuracy: 0.7504 - 

Lets try some modification in model

In [36]:
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import f1_score
class F1ScoreCallback(Callback):
    def __init__(self, validation_data):
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs={}):
        x_val, y_val = self.validation_data
        y_pred = self.model.predict(x_val)
        y_pred_class = np.argmax(y_pred, axis=1)
        score = f1_score(y_val, y_pred_class, average='macro')
        print(f'Epoch {epoch+1}, F1 Score: {score:.4f}')

f1_callback = F1ScoreCallback(validation_data=(val_data_array, val_label_array))


In [37]:
def EEGNetModel_new(input_shape):
    clear_session()
    inputs = Input(shape=input_shape)
    #Block1
    x = Conv1D(8, 64, activation='relu')(inputs)
    x = MaxPooling1D(2)(x)
    x = Dropout(0.5)(x)

    #Block2
    x = Conv1D(8, 64, activation='relu')(x)
    x = MaxPooling1D(2)(x)
    x = Dropout(0.5)(x)

    #Block3
    x = Conv1D(32, 32, activation='relu')(x)
    x = MaxPooling1D(2)(x)
    x = Dropout(0.5)(x)

    #Flatten and Dense Layers
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    #Define the model
    model = Model(inputs = inputs, outputs=outputs)

    #compile
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [41]:
new_model = EEGNetModel_new((2500, 21))
new_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 2500, 21)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 2437, 8)        │        10,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 1218, 8)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1218, 8)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 1155, 8)        │         4,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 577, 8)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 577, 8)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 546, 32)        │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 273, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 273, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8736)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,118,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,145,585 (4.37 MB)

 Trainable params: 1,145,585 (4.37 MB)

 Non-trainable params: 0 (0.00 B)

In [42]:
new_model.fit(X, y, epochs=5, batch_size=32, validation_data=(val_data_array, val_label_array), callbacks=[f1_callback])

Epoch 1/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step  - accuracy: 0.7271 - loss: 
Epoch 1, F1 Score: 0.4287
50/50 ━━━━━━━━━━━━━━━━━━━━ 9s 124ms/step - accuracy: 0.7273 - loss: 0.6039 - val_accuracy: 0.7504 - val_loss: 0.5638
Epoch 2/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/stepp - accuracy: 0.7309 - loss: 
Epoch 2, F1 Score: 0.4287
50/50 ━━━━━━━━━━━━━━━━━━━━ 7s 140ms/step - accuracy: 0.7312 - loss: 0.5894 - val_accuracy: 0.7504 - val_loss: 0.5630
Epoch 3/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/stepp - accuracy: 0.7484 - loss: 
Epoch 3, F1 Score: 0.4287
50/50 ━━━━━━━━━━━━━━━━━━━━ 7s 138ms/step - accuracy: 0.7484 - loss: 0.5735 - val_accuracy: 0.7504 - val_loss: 0.5730
Epoch 4/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/stepp - accuracy: 0.7275 - loss: 
Epoch 4, F1 Score: 0.4287
50/50 ━━━━━━━━━━━━━━━━━━━━ 7s 138ms/step - accuracy: 0.7278 - loss: 0.5866 - val_accuracy: 0.7504 - val_loss: 0.5642
Epoch 5/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/stepp - accuracy: 0.7469 - loss:
Epoch 5, F1 Score: 0.4287
50/50 ━━